In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler

In [2]:
tickers = ["ARL","BH","CIX","FDBC","GEF","NATH","NKSH","NWFL","PLBC","PNRG","QRTEB","RGCO","STRS","SWKH","TCI","VABK","VALU"]

## Part 1 deal with shared data

In [3]:
# Read the CSV file
data = pd.read_csv('train_data_0_20.csv',low_memory=False)
data['date'] = pd.to_datetime(data['date'], format='%Y/%m/%d')
data.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,sentiment
0,2015-10-01,ARL,7.030000,7.030000,7.030000,7.030000,1300,3,0.143887,7.711643,6.701357,56.887952,-9.827596,18.536363,7.094667,6.268500,22.549999,0.0,0
1,2015-10-01,BH,243.279999,245.199997,238.273331,241.353332,8850,3,-4.967863,254.549970,240.808029,40.288793,-150.888171,28.459297,249.845555,267.082889,22.549999,0.0,0
2,2015-10-01,CIX,10.990000,10.990000,10.830000,7.353430,200,3,-0.012619,7.685737,7.304256,45.040564,-93.320721,20.168078,7.473885,7.577119,22.549999,0.0,0
3,2015-10-01,FDBC,23.333332,23.333332,23.333332,18.255781,0,3,0.222549,18.046982,16.483571,73.236650,320.645967,36.525050,17.178692,17.132404,22.549999,0.0,0
4,2015-10-01,GEF,32.040001,32.540001,31.040001,22.461304,293500,3,0.044349,25.739347,22.292050,45.435755,-15.270326,5.304819,22.890805,22.948372,22.549999,0.0,0


In [4]:
import ta
# Define the function for calculating ATR and other indicators
def calculate_atr_pc(group):
    # Calculate the ATR
    group['ATR'] = ta.volatility.AverageTrueRange(
        high=group['high'],
        low=group['low'],
        close=group['close'],
        window=14  # Define your window size
    ).average_true_range()

    # Calculate the price change
    group['Price_Change'] = group['close'].diff()

    return group

# Apply the function to each group
data = data.groupby('tic', group_keys=False).apply(calculate_atr_pc)



# Filter out rows where ATR is zero (optional)
data = data[data['ATR'] != 0]

# Display the updated dataset with ATR
data.tail()


,date,tic,open,high,low,close,volume,day,macd,boll_ub,...,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,sentiment,ATR,Price_Change
38347,2024-09-18,STRS,27.410000,28.309999,26.280001,26.280001,13800,2,0.378013,28.013316,...,51.882147,130.492371,15.690076,25.004000,25.837167,18.23,13.37937,0,1.255661,-1.480000
38348,2024-09-18,SWKH,16.959999,17.299999,16.770000,16.950001,20000,2,-0.057518,17.668155,...,48.225863,-46.888029,9.608028,17.130333,17.090667,18.23,13.37937,0,0.454393,-0.099998
38349,2024-09-18,TCI,28.250000,28.920000,27.620001,27.620001,3000,2,-0.396883,30.630781,...,43.494571,-101.713401,18.301329,29.073333,29.319500,18.23,13.37937,0,0.822697,-0.869999
38350,2024-09-18,VABK,40.990002,41.209999,40.180000,41.040001,13800,2,1.288482,42.184776,...,62.731284,104.565840,28.385168,37.864954,36.354800,18.23,13.37937,0,1.292221,-0.160000
38351,2024-09-18,VALU,42.880001,44.700001,42.490002,44.700001,12000,2,0.194193,44.466663,...,53.666986,183.094744,7.058237,42.489000,43.618247,18.23,13.37937,0,1.791891,2.000000


In [5]:
# Assuming df_scaled is already prepared with 'date' and 'month' columns
df_scaled = pd.DataFrame(data)
df_scaled['date'] = pd.to_datetime(df_scaled['date'], format='%Y/%m/%d')

# Extract the month from the 'date' column
df_scaled['month'] = df_scaled['date'].dt.to_period('M')
df_scaled['sentiment'] = pd.to_numeric(df_scaled['sentiment'], errors='coerce')

# List of features to scale (exclude 'sentiment' since it's non-numeric)
features_scaled = [
    'close',
    'volume',
    'macd',
    'boll_ub',
    'boll_lb',
    'rsi_30',
    'cci_30',
    'dx_30',
    'close_30_sma',
    'close_60_sma',
    'turbulence',
    'vix',
    'sentiment',
    'ATR',
    'Price_Change'
]


# Drop any rows with NaN values in the features (Optional if needed)
df_scaled = df_scaled.fillna(0)

# Create a StandardScaler object
scaler = StandardScaler()

# Scale the selected features and store them in a new DataFrame
df_scaled_features = pd.DataFrame(scaler.fit_transform(df_scaled[features_scaled]), columns=features_scaled)

# Combine the scaled features with the non-scaled columns (e.g., 'date', 'tic', 'month')
df_final = pd.concat([df_scaled[['date', 'tic', 'month']], df_scaled_features], axis=1)
df_final.dropna(inplace=True)

# Display the first few rows of the final DataFrame
df_final.head()

,date,tic,month,close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,vix,sentiment,ATR,Price_Change
221,2015-10-20,ARL,2015-10,-0.765079,-0.165433,-0.279741,-0.738759,-0.772951,-1.237234,-0.603321,-0.237547,-0.746535,-0.735251,-0.322216,-0.545894,0.066251,-1.248227,0.028909
222,2015-10-20,BH,2015-10,4.970078,-0.150340,1.081444,4.732676,5.041078,-0.173815,0.558980,0.520486,4.858381,4.940168,-0.322216,-0.545894,0.066251,1.064206,0.117742
223,2015-10-20,CIX,2015-10,-0.720587,-0.175494,-0.023806,-0.722450,-0.702156,-0.396184,-0.478956,-0.671993,-0.716218,-0.715017,-0.322216,-0.545894,0.066251,-0.201969,-0.005259
224,2015-10-20,FDBC,2015-10,-0.460794,-0.156629,0.102387,-0.494830,-0.450392,2.439890,1.663509,1.561874,-0.475210,-0.483095,-0.322216,-0.545894,0.066251,0.215591,0.062464
225,2015-10-20,GEF,2015-10,-0.351318,1.898014,-0.099963,-0.330911,-0.312719,-0.457348,-0.715204,-0.802423,-0.328932,-0.341927,-0.322216,-0.545894,0.066251,1.740675,-0.245632


## Part 2 add new data (market and risk free rate)

In [6]:
start_date="2011-10-01"
end_date="2024-09-30"
window_size = 252

In [7]:
column_names = ['date','tic','stat_Anomaly_Probability']
final_result_df = pd.DataFrame(columns=column_names)

In [8]:
for target_ticker in tickers:
    treasury_data =yf.download('^IRX', start=start_date, end=end_date)
    treasury_data['daily_risk_free_rate'] = (treasury_data['Adj Close'] / 100) / 252
    treasury_data=treasury_data[['daily_risk_free_rate']]
    mkt_data =yf.download('^RUT', start=start_date, end=end_date)
    mkt_data['daily_market_return'] = mkt_data['Adj Close'].pct_change()
    mkt_data=mkt_data[['daily_market_return']]
    stock_data =yf.download(target_ticker, start=start_date, end=end_date)
    stock_data['daily_return'] = stock_data['Adj Close'].pct_change()
    stock_data=stock_data[['daily_return']]
    combined_df = pd.concat([treasury_data, mkt_data, stock_data], axis=1).dropna()
    combined_df["tic"]=target_ticker
    combined_df["r_minus_rf"]=combined_df["daily_return"]-combined_df["daily_risk_free_rate"]
    combined_df["rm_minus_rf"]=combined_df["daily_market_return"]-combined_df["daily_risk_free_rate"]
    combined_df = combined_df.dropna()
    combined_df_reset = combined_df.reset_index()
    combined_df_reset.rename(columns={'Date':'date'}, inplace=True)
    res_df=combined_df_reset[['date','tic','r_minus_rf','rm_minus_rf']]
    shared_data=df_final[df_final['tic']==target_ticker].sort_values(by='date')
    df_merged = pd.merge(shared_data, res_df, on=['date', 'tic'],how='left')
    df_merged.ffill(inplace=True)
    X = df_merged[['rm_minus_rf','volume', 'macd',
       'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma',
       'close_60_sma', 'vix', 'turbulence', 'sentiment', 'ATR', 'Price_Change']]
    y = df_merged['r_minus_rf']
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    df_merged['residual']=model.resid
    res_std=df_merged['residual'].std()
    df_merged['stat_Anomaly_Probability']=2*np.abs(norm.cdf(df_merged['residual']/res_std, 0, 1)-0.5)
    answer_df=df_merged[['date','tic','stat_Anomaly_Probability']]
    final_result_df = pd.concat([final_result_df, answer_df], ignore_index=True)
final_result_df =final_result_df.sort_values(by=['date', 'tic'])
final_result_df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
C:\Users\hc\AppData\Local\Temp\ipykernel_18252\1112539657.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_result_df = pd.concat([final_result_df, answer_df], ignore_index=True)
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********

,date,tic,stat_Anomaly_Probability
0,2015-10-20,ARL,0.615856
2230,2015-10-20,BH,0.228575
4460,2015-10-20,CIX,0.623765
6690,2015-10-20,FDBC,0.065948
8920,2015-10-20,GEF,0.178754
...,...,...,...
28989,2024-08-29,STRS,0.941004
31219,2024-08-29,SWKH,0.176823
33449,2024-08-29,TCI,0.234226
35679,2024-08-29,VABK,0.895677
